**Connect to Snowflake**

In [ ]:
# Install snowflake connector: pip install snowflake-connector-python
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector as snow
import pandas as pd
import re
import os
from tqdm import tqdm
import numpy as np

# Be aware that MFA will be triggered
print("Connecting, please check for any MFA notifications.")

# Create a snowflake connection
conn = snow.connect(
user="username", # Replace with your username
password="password", # Replace with your password
account="account",
warehouse="DEFAULT_WAREHOUSE",
database="CLASS_DATABASE",
schema="ANALYSIS")

ModuleNotFoundError: No module named 'snowflake'

**Data Pre-Processing with SQL**
1. Drop conversation start with OUT direction (i.e. Agent to User messages)
2. Drop conversation with one direction (i.e. conversations that were initiated but never continued)

In [2]:
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS CLEAN_DATA_Final as( 
WITH first_messages AS (
    -- Identify the first message in each conversation
    SELECT 
        CONVERSATION_ID,
        MESSAGE_ID,
        MESSAGE,
        MESSAGE_DIRECTION,
        SENT_AT,
        ROW_NUMBER() OVER (PARTITION BY CONVERSATION_ID ORDER BY SENT_AT ASC) AS row_num
    FROM class_shared_data.class_dataset.message_dataset
),
conversations_starts_with_OUT AS (
    -- Select conversations where the first message is OUT-bound
    SELECT CONVERSATION_ID
    FROM first_messages
    WHERE MESSAGE_DIRECTION = 'Out' AND row_num = 1
),
conversations_with_one_direction AS (
    -- Select conversations with only one message direction (IN or OUT)
    SELECT CONVERSATION_ID
    FROM class_shared_data.class_dataset.message_dataset
    GROUP BY CONVERSATION_ID
    HAVING COUNT(DISTINCT MESSAGE_DIRECTION) = 1
),

conversations_to_remove AS (
    -- Combine conversations to be removed based on above criteria
    SELECT CONVERSATION_ID FROM conversations_with_one_direction
    UNION
    SELECT CONVERSATION_ID FROM conversations_starts_with_OUT
),

filtered_messages AS (
    -- Filter messages that are not part of conversations to be removed
    SELECT *
    FROM class_shared_data.class_dataset.message_dataset
    WHERE CONVERSATION_ID NOT IN (
        SELECT CONVERSATION_ID
        FROM conversations_to_remove
    )
),

messages_with_directions AS (
    -- Concatenate MESSAGE_DIRECTION and MESSAGE for aggregation
    SELECT 
        CONVERSATION_ID,
        CONCAT(MESSAGE_DIRECTION, ':.,', MESSAGE) AS formatted_message,
        SENT_AT
    FROM filtered_messages
),

aggregated_messages AS (
    -- Aggregate the messages in order of SENT_AT for each conversation
    SELECT 
        CONVERSATION_ID,
        MIN(SENT_AT) AS SENT_AT,
        LISTAGG(formatted_message, '\n') WITHIN GROUP (ORDER BY SENT_AT) AS conversation_messages
    FROM messages_with_directions
    GROUP BY CONVERSATION_ID
)

SELECT 
    conv.CONVERSATION_ID,
    agg.conversation_messages, 
    conv.CONVERSATION_CHANNEL,
    conv.AGENT_LABELED_CATEGORY,
    conv.AGENT_LABELED_APPLICATION,
    conv.USER_LABELED_CATEGORY,
    conv.TRIAGE_MESSAGES,
    agg.SENT_AT    
FROM class_shared_data.class_dataset.conversation_dataset AS conv
JOIN aggregated_messages AS agg
    ON conv.CONVERSATION_ID = agg.CONVERSATION_ID)
""")

In [3]:
# Read the data from the table created above
cur = conn.cursor()
cur.execute("""
select * from CLEAN_DATA_Final   -- Match the table name with the one you created
""")

# Fetch all the results from the query
results = cur.fetchall()

# Get the column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Convert the results into DataFrame
combined_data = pd.DataFrame(results, columns=column_names)

**Check the Data**

In [4]:
combined_data.head()

,CONVERSATION_ID,CONVERSATION_MESSAGES,CONVERSATION_CHANNEL,AGENT_LABELED_CATEGORY,AGENT_LABELED_APPLICATION,USER_LABELED_CATEGORY,TRIAGE_MESSAGES,SENT_AT
0,4bc9d6ac597672e3801d6160a03d62cae1f3ac07,"In:., From: HIDDEN_EMAIL Date: Tue, 25 J...",email,None,None,None,None,2022-01-25 19:20:54.967 Z
1,31b582d4e3156a062479bd96a660a269b19dcad3,"In:.,Below is the one-time password for loggin...",email,None,None,non_request_interaction,None,2021-09-08 13:05:50.000 Z
2,7ff7673238c978597a38a1a3226ee3c2ebe2a361,"In:.,hi there! can you please send me a secure...",chat,None,None,sysadmin_task.password_reset,None,2022-01-03 18:38:05.518 Z
3,8c59b90503261f88dc8302408a3b2012b307723b,"In:.,We had a new hire start on Thursday. I wa...",chat,workstation.provision.new_provision,None,None,[SYSTEM MESSAGE] **Request overview**\n ...,2022-02-02 18:23:03.154 Z
4,cb95a74dbeb7e18fd4ce57fd1ffd7882e77d3765,"In:.,Hi! I'm trying to register my laptops wit...",chat,workstation.mdm_policy.enroll_device,None,None,[SYSTEM MESSAGE] **Request overview**\n ...,2022-07-21 16:37:23.071 Z


In [5]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484034 entries, 0 to 484033
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   CONVERSATION_ID            484034 non-null  object
 1   CONVERSATION_MESSAGES      484034 non-null  object
 2   CONVERSATION_CHANNEL       484034 non-null  object
 3   AGENT_LABELED_CATEGORY     380687 non-null  object
 4   AGENT_LABELED_APPLICATION  126326 non-null  object
 5   USER_LABELED_CATEGORY      72154 non-null   object
 6   TRIAGE_MESSAGES            215792 non-null  object
 7   SENT_AT                    484034 non-null  object
dtypes: object(8)
memory usage: 29.5+ MB


In [6]:
# Check NAs
combined_data.isna().sum()

CONVERSATION_ID                   0
CONVERSATION_MESSAGES             0
CONVERSATION_CHANNEL              0
AGENT_LABELED_CATEGORY       103347
AGENT_LABELED_APPLICATION    357708
USER_LABELED_CATEGORY        411880
TRIAGE_MESSAGES              268242
SENT_AT                           0
dtype: int64

**Keep Only the Initial Messages from User**
1. Remove any messages starting from the agent's first response (i.e. messages that come after "Out:.,")
2. Delete "In:.," from remaining messages (because all of them now should be In messages)
3. Remove the conversations with less than 10 words (to remove conversations with only "Hello", "Hi", etc.)

In [7]:
# Remove anything that comes after "Out:.,"
combined_data['CONVERSATION_MESSAGES'] = combined_data['CONVERSATION_MESSAGES'].str.split('Out:.,').str[0]

# Delete "In:" 
combined_data['CONVERSATION_MESSAGES'] = combined_data['CONVERSATION_MESSAGES'].str.replace('In:.,', '')

# Replace '\n， \r' with ''
def remove_single_letter_backslash(text):
    return re.sub(r'\n|\r', ' ', text)
combined_data['CONVERSATION_MESSAGES'] = combined_data['CONVERSATION_MESSAGES'].apply(remove_single_letter_backslash)

# Remove conversations with less than 10 words
combined_data_10 = combined_data[combined_data['CONVERSATION_MESSAGES']
           .str.replace(r'[^\w\s]', '', regex=True)  
           .str.split()
           .str.len() >= 10]

print('Shape of combined_data: ', combined_data.shape)
print('Shape of combined_data_10: ', combined_data_10.shape)

Shape of combined_data:  (484034, 8)
Shape of combined_data_10:  (351087, 8)


**Add Simplified Category (i.e. Automatable Categories + Other (Not Automatable))**

In [ ]:
# Read Excel file that contains simplified categories
df_ticket_category = pd.read_excel('simplified_categories.xlsx')

# remain column: AGENT_LABELED_CATEGORY, SIMPLIFIED_CATEGORY
df_ticket_category = df_ticket_category[['AGENT_LABELED_CATEGORY', 'SIMPLIFIED_CATEGORY']]

# fill in missing values in SIMPLIFIED_CATEGORY as OTHER
df_ticket_category['SIMPLIFIED_CATEGORY'] = df_ticket_category['SIMPLIFIED_CATEGORY'].fillna('OTHER')

In [10]:
df_ticket_category['SIMPLIFIED_CATEGORY'].value_counts()

SIMPLIFIED_CATEGORY
OTHER                                              201
UNKNOWN                                              5
os_update                                            2
add_user_to_channel OR remove_user_from_channel      2
add_license OR remove_license                        2
shipping_request                                     2
add_user_to_group OR remove_user_from_group          2
add_user OR remove_user                              1
password_reset                                       1
reset_mfa                                            1
Name: count, dtype: int64

In [11]:
# Convert UNKNOWN to OTHER
df_ticket_category['SIMPLIFIED_CATEGORY'] = df_ticket_category['SIMPLIFIED_CATEGORY'].replace('Unknown', 'OTHER')

# Replace the AGENT_LABELED_CATEGORY in df_10 with SIMPLIFIED_CATEGORY in df_ticket
combined_data_10 = pd.merge(combined_data_10, df_ticket_category, on='AGENT_LABELED_CATEGORY', how='left')

In [12]:
combined_data_10.head()

,CONVERSATION_ID,CONVERSATION_MESSAGES,CONVERSATION_CHANNEL,AGENT_LABELED_CATEGORY,AGENT_LABELED_APPLICATION,USER_LABELED_CATEGORY,TRIAGE_MESSAGES,SENT_AT,SIMPLIFIED_CATEGORY
0,4bc9d6ac597672e3801d6160a03d62cae1f3ac07,"From: HIDDEN_EMAIL Date: Tue, 25 Jan 20...",email,None,None,None,None,2022-01-25 19:20:54.967 Z,NaN
1,31b582d4e3156a062479bd96a660a269b19dcad3,Below is the one-time password for logging int...,email,None,None,non_request_interaction,None,2021-09-08 13:05:50.000 Z,NaN
2,7ff7673238c978597a38a1a3226ee3c2ebe2a361,hi there! can you please send me a secure link...,chat,None,None,sysadmin_task.password_reset,None,2022-01-03 18:38:05.518 Z,NaN
3,8c59b90503261f88dc8302408a3b2012b307723b,We had a new hire start on Thursday. I was wor...,chat,workstation.provision.new_provision,None,None,[SYSTEM MESSAGE] **Request overview**\n ...,2022-02-02 18:23:03.154 Z,OTHER
4,cb95a74dbeb7e18fd4ce57fd1ffd7882e77d3765,Hi! I'm trying to register my laptops with ele...,chat,workstation.mdm_policy.enroll_device,None,None,[SYSTEM MESSAGE] **Request overview**\n ...,2022-07-21 16:37:23.071 Z,OTHER


In [13]:
combined_data_10 = combined_data_10[['CONVERSATION_ID','CONVERSATION_CHANNEL', 'AGENT_LABELED_APPLICATION','TRIAGE_MESSAGES', 'AGENT_LABELED_CATEGORY', 'SIMPLIFIED_CATEGORY', 'CONVERSATION_MESSAGES']]

**Extract Relevant Data from TRIAGE_MESSAGES**

In [14]:
combined_data_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351087 entries, 0 to 351086
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   CONVERSATION_ID            351087 non-null  object
 1   CONVERSATION_CHANNEL       351087 non-null  object
 2   AGENT_LABELED_APPLICATION  89608 non-null   object
 3   TRIAGE_MESSAGES            143870 non-null  object
 4   AGENT_LABELED_CATEGORY     262628 non-null  object
 5   SIMPLIFIED_CATEGORY        262628 non-null  object
 6   CONVERSATION_MESSAGES      351087 non-null  object
dtypes: object(7)
memory usage: 18.8+ MB


In [15]:
TRIAGE_message = combined_data_10['TRIAGE_MESSAGES']

In [16]:
# Check TRIAGE_message format types
TRIAGE_message_first_70 = TRIAGE_message.str[:70]
TRIAGE_message_first_70.value_counts()

TRIAGE_MESSAGES
[SYSTEM MESSAGE] **Request overview**\n                \n**Request type*      117556
[SYSTEM MESSAGE] **Request overview**\n                \n**What can we h       24219
[SYSTEM MESSAGE]‎‎\n                \n\n**Request overview**\n                  2094
[SYSTEM MESSAGE] **Request ******************** **********************             1
Name: count, dtype: int64

In [17]:
# First type of triage message 
# extract information we need 
print(combined_data_10[combined_data_10['CONVERSATION_ID'] == '2a3025b0bc2acdbc5706282a64e87be1193c8768']['TRIAGE_MESSAGES'].values[0])
# Useful information:  Request type, Additiuonal details.

[SYSTEM MESSAGE] **Request overview**
                
**Request type**
 **                
**Are those affected able to work?**
 **                
**Additional details**
 need ********** name and password to use zoom


In [18]:
# Second type of triage message
print(combined_data_10[combined_data_10['CONVERSATION_ID'] == '23ba5c0004b4e6c230290e1f599d25e87f2ff0e0']['TRIAGE_MESSAGES'].values[0])
# Useful information: What can we help you with?, What do you need assistance with?, Additional details.

[SYSTEM MESSAGE] **Request overview**
                
**What can we help you with?**
 Something is not working
                
**How many users need assistance?**
 One User
                
**Can users currently work?**
 Work is not ******** or blocked
                
**What do you need assistance with?**
 Login issue
                
**Additional details**
 i just reset the computer for a new user and i am still seeing the old username log in


In [ ]:
def extract_info(text):
    if text is None:
        return ''
    
    if text == '':
        return ''
    
    # Extract from the first type
    if '**Request type**' in text:
        request_type_match = re.search(r'\*\*Request type\*\*\n (.+)', text)
        additional_details_match = re.search(r'\*\*Additional details\*\*\n (.+)', text)
        request_type = request_type_match.group(1) if request_type_match else ''
        additional_details = additional_details_match.group(1) if additional_details_match else ''
        return f"My request type is {request_type}, There are some additional detail which is {additional_details}"
    
    # Extract from the second type
    elif '**What can we help you with?**' in text:
        help_match = re.search(r'\*\*What can we help you with\?\*\*\n (.+)', text)
        assistance_match = re.search(r'\*\*What do you need assistance with\?\*\*\n (.+)', text)
        additional_details_match = re.search(r'\*\*Additional details\*\*\n (.+)', text)
        help_info = help_match.group(1) if help_match else ''
        assistance_info = assistance_match.group(1) if assistance_match else ''
        additional_details = additional_details_match.group(1) if additional_details_match else ''
        return f"I want you to help me with {help_info}. I want you to assist me with {assistance_info}. There are some additional details that {additional_details}"
    
    return ''

# Apply the function to the DataFrame
combined_data_10['Extracted_TRIAGE'] = combined_data_10['TRIAGE_MESSAGES'].apply(extract_info)

In [ ]:
# Combine Extracted_TRIAGE and CONVERSATION_MESSAGES
combined_data_10['MESSAGE_PLUS_TRIAGE'] = combined_data_10['Extracted_TRIAGE'] + ' ' + combined_data_10['CONVERSATION_MESSAGES']

**Upload Final Dataset to Snowflake**

In [ ]:
# Match the table name with the name you created in the beginning
write_pandas(conn, combined_data_10, "CLEAN_DATA_10WORDS_FINAL", auto_create_table=True)

(True,
 1,
 351087,
 [('iqttpyrsph/file0.txt',
   'LOADED',
   351087,
   351087,
   1,
   0,
   None,
   None,
   None,
   None)])

**Get Snowflake Embedding (multilingual-e5-large)**

In [22]:
cur = conn.cursor()
cur.execute("""
CREATE TABLE SNOWFLAKE_EMBEDDING_TRIAGE_FINAL AS
select CONVERSATION_ID, SIMPLIFIED_CATEGORY,CONVERSATION_CHANNEL,MESSAGE_PLUS_TRIAGE ,
SNOWFLAKE.CORTEX.EMBED_TEXT_1024( 'multilingual-e5-large', MESSAGE_PLUS_TRIAGE ) as multilingual_e5large_embedding,
from CLEAN_DATA_10WORDS_Final
""")